<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

## Lab 4.2.1: Feature Selection

### 1. Load & Explore Data

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

#### 1.1 Load Data

In [57]:
# Read CSV
wine_csv = pd.read_csv('winequality_merged.csv')

#### 1.2 Explore Data (Exploratory Data Analysis)

In [58]:
# ANSWER
wine_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  red_wine              6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB


In [60]:
wine_csv.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378,0.246114
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255,0.430779
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000,0.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000,0.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000,0.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000,0.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000,1.000000


### 2. Set Target Variable

Create a target variable for wine quality.

In [61]:
# Target Variable
wine_quality = wine_csv.quality

### 3. Set Predictor Variables

Create a predictor matrix with variables of your choice. State your reason.

In [62]:
# ANSWER
wine_corr=wine_csv.corr() # finding correlation of the dataset
print(wine_corr.quality)  # correlation column with quality

fixed acidity          -0.076743
volatile acidity       -0.265699
citric acid             0.085532
residual sugar         -0.036980
chlorides              -0.200666
free sulfur dioxide     0.055463
total sulfur dioxide   -0.041385
density                -0.305858
pH                      0.019506
sulphates               0.038485
alcohol                 0.444319
quality                 1.000000
red_wine               -0.119323
Name: quality, dtype: float64


In [63]:
wine_corr['quality_corr_abs'] = np.abs(wine_corr['quality']) # duplicating quality column with absolute number to a new column
wine_corr.quality_corr_abs.sort_values(ascending=False)      # sorting and choose top 3 for correlation to quality 

quality                 1.000000
alcohol                 0.444319
density                 0.305858
volatile acidity        0.265699
chlorides               0.200666
red_wine                0.119323
citric acid             0.085532
fixed acidity           0.076743
free sulfur dioxide     0.055463
total sulfur dioxide    0.041385
sulphates               0.038485
residual sugar          0.036980
pH                      0.019506
Name: quality_corr_abs, dtype: float64

In [64]:
predictor_x = wine_csv[['alcohol', 'density','volatile acidity']] # creating the predictor variable
predictor_x

,alcohol,density,volatile acidity
0,9.4,0.99780,0.70
1,9.8,0.99680,0.88
2,9.8,0.99700,0.76
3,9.8,0.99800,0.28
4,9.4,0.99780,0.70
...,...,...,...
6492,11.2,0.99114,0.21
6493,9.6,0.99490,0.32
6494,9.4,0.99254,0.24
6495,12.8,0.98869,0.29


### 4. Using Linear Regression Create a Model and Test Score

In [65]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [66]:
# Train-Test Split

predictor_x_train, predictor_x_test, wine_quality_train, wine_quality_test = train_test_split(predictor_x, wine_quality, test_size=0.20, random_state=42)

In [67]:
# Create a model for Linear Regression
line_reg = LinearRegression()

# Fit the model with the Training data
line_reg.fit(predictor_x_train, wine_quality_train)
# Calculate the score (R^2 for Regression) for Training Data
print(line_reg.score(predictor_x_train,wine_quality_train))
# Calculate the score (R^2 for Regression) for Testing Data
print(line_reg.score(predictor_x_test,wine_quality_test))

0.27078026676393185
0.2519667708304062


### BONUS: Cross validation

In [78]:
# Cross validation 
from sklearn.model_selection import KFold 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [79]:
# Set up 5-fold cross validation  
k_fold = KFold(5, shuffle=True)
train_scores = []
train_rmse = []
test_scores = []
test_rmse = []
X = predictor_x[['alcohol']]

for k, (train, test) in enumerate(k_fold.split(X)):
    linear_regr = LinearRegression()
    #print('train: %s, \ntest: %s' % (predictor_x_train, predictor_x_test))
    #print(k, (train,test))
    #print(predictor_x.iloc[train])
    # Get training and test sets for X and y
    # Fit model with training set
    linear_regr.fit(predictor_x.iloc[train], wine_quality.iloc[train])
    
    # Make predictions with training and test set
    train_pred = line_reg.predict(predictor_x.iloc[train])
    test_pred = line_reg.predict(predictor_x.iloc[test])
    # Score R2 and RMSE on training and test sets and store in list
    r2_train = r2_score(wine_quality.iloc[train], train_pred)
    r2_test = line_reg.score(predictor_x.iloc[test], wine_quality.iloc[test])
# Create a metrics_df dataframe to display r2 and rmse scores
    rmse_train = mean_squared_error(wine_quality.iloc[train], train_pred)
    rmse_test = mean_squared_error(wine_quality.iloc[test], test_pred)

    train_scores.append(r2_train)
    train_rmse.append(rmse_train)
    test_scores.append(r2_test)
    test_rmse.append(rmse_test)

In [81]:
# Describe the metrics
df_dict ={
    'train_score': train_scores,
    'train_rmse' : train_rmse,
    'test_Scores': test_scores,
    'test_rmse' : test_rmse
}

df_scores = pd.DataFrame(df_dict)
df_scores

,train_score,train_rmse,test_Scores,test_rmse
0,0.266758,0.554916,0.262167,0.574006
1,0.262471,0.567467,0.286130,0.523828
2,0.270100,0.552489,0.255960,0.583733
3,0.264245,0.562076,0.276708,0.545370
4,0.271929,0.556731,0.247928,0.566756


### 5. Feature Selection

What's your score (R^2 for Regression) for Testing Data? 

How many feature have you selected? Can you improve your score by selecting different features?

In [52]:
def func_line_model(df,predictor,target='quality'):
    X = df[predictor]
    y = df[target]
    func_regr = LinearRegression()
    func_regr.fit(X,y)
    return func_regr,X,y

In [54]:
alcohol_regr, alcohol_X, alcohol_y = func_line_model(wine_csv,['alcohol'])
alcohol_regr, alcohol_X, alcohol_y
alcohol_regr.score(alcohol_X,alcohol_y)

0.1974189472217911

**Please continue with Lab 4.2.2 with the same dataset.**



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



